# SQLite

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')

In [ ]:
conn.execute(
    """
    CREATE TABLE students (
        name VARCHAR(50) NOT NULL,
        age INT NOT NULL
    );
""")

In [ ]:
values = ('Bill', '30')

conn.execute("INSERT INTO students VALUES (?, ?)", values)

In [ ]:
cursor = conn.execute('SELECT * FROM students')
cursor.fetchall() # .fetchone()


In [ ]:
conn.execute("""UPDATE students SET age = ? WHERE name = ?""", (25, 'Bill'))

# PostgresSQL 

1. Server based
2. Data types
3. Full text search (LIKE requests)

## install engine locally

> pip install psycopg2

In [ ]:
import psycopg2

connection = psycopg2.connect(
    host='localhost',
    database='postgres',
    user='user',
    password='password',
    port=5432,
)

In [ ]:
cursor = connection.cursor()

cursor.execute("""SELECT * FROM student""")

In [ ]:
cursor.fetchall()

In [ ]:
cursor.close()

In [ ]:
with connection.cursor() as curs:
    curs.execute("""SELECT * FROM student""")

    print(curs.fetchall())

In [ ]:
# Update and commit changes

with connection.cursor() as curs:
    curs.execute("""SELECT count(*) FROM student;""")
    print(curs.fetchone())

    curs.execute("INSERT INTO student(name, age) VALUES (%s, %s)", ('STUDENT NAME 2', 35))
    
    curs.execute("""SELECT count(*) FROM student;""")
    print(curs.fetchone())


## SQLAlchemy

> pip install SQLAlchemy

### What does SQLAlchemy do?

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class Student(Base):
    __tablename__ = 'student'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    def __str__(self):
        return f'This is {self.id} student {self.name}. Age: {self.age}'

    def __repr__(self):
        return f'This is {self.id} student {self.name}. Age: {self.age}'


In [ ]:
from sqlalchemy import create_engine

DATABASE_URI = 'postgresql://{user}:{password}@{host}:{port}/{database}'


engine = create_engine(
    DATABASE_URI.format(
        host='localhost',
        database='postgres',
        user='user',
        password='password',
        port=5432,
    )
)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()

In [ ]:
session.close()

In [ ]:
first_student = session.query(Student).first() # select * from student limit 1

In [ ]:
first_student

In [ ]:
first_student.id

In [ ]:
students = session.query(Student).all() # select * from student 

for student in students:
    print(f'Student id  {student.id}')
    print(f'Student name  {student.name}')
    print(f'Student age  {student.age}')
    print()

In [ ]:
session.query(Student.name, Student.age).all()
# Select name, age from student

In [ ]:
first_old_student = session.query(Student).filter(Student.age > 20).first()
# Select * from student where age > 20 limit 1;
print(first_old_student)

In [ ]:
first_old_student = session.query(Student).filter(Student.age > 20).order_by(Student.id.desc()).first()
# Select * from student where age > 20 order by id desc limit 1;
print(first_old_student)

In [ ]:
old_students: list[Student] = session.query(Student).filter(Student.age > 20).order_by(Student.id.desc()).all()
# Select * from student where age > 20;
print(old_students)

In [ ]:
old_students = session.query(Student).filter(Student.age > 20).order_by(Student.id.asc()).all()
# Select * from student where age > 20;
print(old_students)

### AND / OR

In [ ]:
mature_students: list[Student] = (
    session.query(Student)
    .filter(Student.age.between(18, 20))
    .all()
)
# Select * from student where age between 18 and 20

print(mature_students)

In [ ]:
### WHERE 

from sqlalchemy import and_

mature_students: list[Student] = (
    session.query(Student)
    .filter(
        and_(
            Student.age.between(18, 20),
            Student.name == 'Iria',
        )
    ).all()
)

# SELECT * from student where
# age between 18 and 20 and nama = 'Iria'
#

print(mature_students)

In [ ]:
from sqlalchemy import or_

session.query(Student).filter(
    or_(
        Student.name == 'Bae',
        Student.name == 'Lila',
    )
).all()

# select * from student 
# where name = 'bae' or name = 'Lila'

In [ ]:
session.query(Student).order_by(Student.name.desc()).limit(2).all()

# select * from student order by name desc limit 2;

In [ ]:
session.query(Student).limit(2).all()

In [ ]:
from sqlalchemy import and_

(
    session.query(Student)
    .filter(
        and_(
            or_(
                Student.age < 18,
                Student.age > 25,
            ),
            Student.id.between(1, 10)
        )
    )
    .order_by(Student.name.desc())
    .limit(10)
    .all()
)


# Select * from student where (age < 18 or age > 25) and id between 1 and 10 order by name desc limit 10

## Update

In [ ]:
from random import randint

bae = session.query(Student).filter(Student.name == 'Bae').first()

print(bae)
print(bae.id)

In [ ]:

bae.age = randint(18, 25)

print(bae)

In [ ]:
bae.age

In [ ]:
bae = session.query(Student).filter(Student.name == 'Bae').first()
bae

In [ ]:

session.commit()
# session.rollback()

In [ ]:
bae

In [ ]:
bae = session.query(Student).filter(Student.name == 'Bae').first()
bae

## Insert

In [ ]:
new_student = Student(name='Maksym', age=25)

session.add(new_student)

In [ ]:
session.query(Student).filter(
    and_(
        Student.name == 'Maksym',
        Student.age == 25,
    )
).all()

In [ ]:
session.commit()

In [ ]:
from random import randint, choice

random_names = ['Joe', 'Winston', 'Benedict', 'Andrea', 'Fillipe', 'Iria', 'Eric', 'Joan', 'Pablo', 'Brian', 'Timothy', 'Lee']

for _ in range(10):
    student = Student(name=choice(random_names), age=randint(18, 25))
    session.add(student)

session.commit()

In [ ]:
session.query(Student).all()

## Update

In [ ]:
last_student = session.query(Student).order_by(Student.id.desc()).first()

last_student

In [ ]:
type(last_student)

In [ ]:
last_student

In [ ]:
session.commit()

## Practice

1. Add models for student, subject and student_subject from previous lessons in SQLAlchemy.
2. Find all students` name that visited 'English' classes.
3. (Optional): Rewrite queries from the previous lesson using SQLAlchemy.